In [1]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)

In [2]:
sample_size = 6000000 #--> 100 k

# question_answer_view = '''
#                 SELECT id AS user_id, reputation, asked, answered
#                         FROM `bigquery-public-data.stackoverflow.users` users
#                         LEFT JOIN(
#                             SELECT owner_user_id AS user_id, COUNT(*) AS asked
#                             FROM `bigquery-public-data.stackoverflow.posts_questions`
#                             GROUP BY user_id
#                         ) questions ON users.id = questions.user_id
#                         LEFT JOIN(
#                             SELECT owner_user_id AS user_id, COUNT(*) AS answered
#                             FROM `bigquery-public-data.stackoverflow.posts_answers`
#                             GROUP BY user_id
#                         ) answers ON users.id = answers.user_id
#                 limit ''' + str(sample_size)


aggregate_query = '''
            SELECT
                reputation AS reputation,
                COUNT(*) AS users,
                SUM(asked) AS questions,
                SUM(answered) AS answers
            FROM(
                SELECT id AS user_id, reputation, asked, answered
                            FROM `bigquery-public-data.stackoverflow.users` users
                            LEFT JOIN(
                                SELECT owner_user_id AS user_id, COUNT(*) AS asked
                                FROM `bigquery-public-data.stackoverflow.posts_questions`
                                GROUP BY user_id
                            ) questions ON users.id = questions.user_id
                            LEFT JOIN(
                                SELECT owner_user_id AS user_id, COUNT(*) AS answered
                                FROM `bigquery-public-data.stackoverflow.posts_answers`
                                GROUP BY user_id
                            ) answers ON users.id = answers.user_id
            )
 group by reputation
 '''


aggregate = (
    client.query(aggregate_query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

aggregate.fillna(0)

,reputation,users,questions,answers
0,1071,209,3316.0,4319.0
1,637,392,5119.0,5494.0
2,891,287,3626.0,4198.0
3,393,972,9927.0,7363.0
4,324,779,5698.0,7967.0
...,...,...,...,...
25291,3560,16,771.0,1161.0
25292,4840,16,505.0,1290.0
25293,3711,16,542.0,1114.0
25294,4336,16,374.0,1707.0


In [3]:

def reputationToCategory(x):
    if(x >= 1 and x <= 100):
        return "Usurpers"
    elif(x > 100 and x <= 1000):
        return "Slaves"
    elif(x > 1000 and x <= 10000):
        return "Lords"
    elif(x > 10000 and x <= 100000):
        return "Grandmasters"
    else: return "Gods"
        
    

aggregate['reputation'] = aggregate['reputation'].map(reputationToCategory)
aggregate

,reputation,users,questions,answers
0,Lords,209,3316.0,4319.0
1,Slaves,392,5119.0,5494.0
2,Slaves,287,3626.0,4198.0
3,Slaves,972,9927.0,7363.0
4,Slaves,779,5698.0,7967.0
...,...,...,...,...
25291,Lords,16,771.0,1161.0
25292,Lords,16,505.0,1290.0
25293,Lords,16,542.0,1114.0
25294,Lords,16,374.0,1707.0


In [4]:
df = aggregate.groupby(['reputation']).sum()

df.to_csv("reputation.csv")

In [5]:
df_plot = pd.read_csv('reputation.csv')

In [6]:
df_plot

,reputation,users,questions,answers
0,Gods,945,82720.0,3126525.0
1,Grandmasters,21352,1543117.0,8071486.0
2,Lords,197310,5279592.0,10234116.0
3,Slaves,813758,6212260.0,5907889.0
4,Usurpers,12533410,7027377.0,3047965.0


In [7]:
def normalizeUsersCount(x):
    totalUsers = df_plot['users'].sum()
    return (x * 100) / totalUsers
def normalizeQuestionsCount(x):
    totalQuestions = df_plot['questions'].sum()
    return (x * 100) / totalQuestions
def normalizeAnswersCount(x):
    totalAnswers = df_plot['answers'].sum()
    return (x * 100) / totalAnswers

In [8]:
df_plot['users'] = df_plot['users'].map(normalizeUsersCount)
df_plot['questions'] = df_plot['questions'].map(normalizeQuestionsCount)
df_plot['answers'] = df_plot['answers'].map(normalizeAnswersCount)
df_plot

,reputation,users,questions,answers
0,Gods,0.006966,0.410622,10.288689
1,Grandmasters,0.157384,7.660025,26.561442
2,Lords,1.454362,26.207866,33.678170
3,Slaves,5.998168,30.837625,19.441532
4,Usurpers,92.383120,34.883862,10.030166


In [52]:
# fig = px.pie(df, values="users", names=df.index, color_discrete_sequence=px.colors.sequential.RdBu)

fig = go.Figure(data=[go.Pie(labels=df_plot["reputation"], values=df_plot["users"], pull=[0.3, 0.2, 0.15, 0.1, 0])])
fig.update_layout(title={
        "x":0.5,
        "y":0.95,
        "text":"Users of Stack Overflow based on reputation categories",
        "xanchor":"center",
        "yanchor":"top"
    })
fig.update_traces(marker=dict(colors=["#c4a15a", "#223943", "#1438de", "#c4a15a", "#007668"]))

fig.show()

In [53]:
##### bar_df = df_plot.iloc[::-1]

colors_answer = ["#223943"] * 5
colors_question = ["#c4a15a"] * 5

bar = go.Figure(data=[
    go.Bar(name='Questions', x=bar_df["reputation"], y=bar_df["questions"], marker_color=colors_question),
    go.Bar(name='Answers', x=bar_df["reputation"], y=bar_df["answers"], marker_color=colors_answer)
])
# Change the bar mode
bar.update_layout(barmode='group', xaxis={'categoryorder':'trace'}, title='Percentage of questions and answers per reputation category',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Percentage %',
        titlefont_size=16,
        tickfont_size=14,
    ), bargroupgap=0.05)
bar.show()